In [ ]:
# Si hacen falta el instalar dvc y git
!pip install dvc
!apt-get update && apt-get install -y git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.2/466.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.8/438.8 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.5/375.5 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/

In [ ]:
import json
import os

PARAMS_FILE = "run_params.json"

class init_param:
    #Se inicializa
    def __init__(self, run_count=0,limpiar_count=0):
        self.run_count = run_count
        self.limpiar_count = limpiar_count
        # Cargar parámetros si el archivo existe
        if os.path.exists(PARAMS_FILE):
            with open(PARAMS_FILE, 'r') as f:
                params = json.load(f)
        else:
            # Inicializar parámetros si el archivo no existe
            params = {"run_count": run_count, "limpiar_count": limpiar_count}
            with open(PARAMS_FILE, 'w') as f:
                json.dump(params, f)
        self.params = params
    # Funcion para obtener parámetros
    def get_params(self):
        return self.params

    #Función para actualizar parámetros
    def update_params(self, params, parametro):
        # Incrementar el contador de corridas
        params[parametro] += 1
        current_run = params[parametro]
        # Guardar los parámetros actualizados
        with open(PARAMS_FILE, 'w') as f:
          json.dump(params, f)
        print(f"Número de corrida actual para {parametro}: {current_run}")

    def init_git(self):
        # Verificar si Git está inicializado si no...
        # Se inicializa el git
        if not os.path.exists('.git'):
            print("Inicializando repositorio Git...")
            !git init
            !git add data/raw.dvc
        else:
            print("Repositorio Git ya inicializado.")

    def init_dvc(self):
        # Verificar si DVC está inicializado
        if not os.path.exists('.dvc'):
            # Si no está inicializado, se inicializa
            print("Inicializando repositorio DVC...")
            !dvc init
            !dvc add data/raw
            !dvc config core.autostage true
            !dvc remote add -d local data/interim
        else:
            print("Repositorio DVC ya inicializado.")

    def commit(self, df, name, contador):
        # Guardar el DataFrame recién modificado
        df.to_csv(name, index=False)
        print(f"Segundo dato modificado guardado en {name}")

        # Usar DVC para agregar el archivo de datos modificado
        !dvc add {name}

        # Usar Git para agregar el archivo .dvc actualizado
        !git add {name}.dvc

        # Confirmar los cambios en Git
        !git commit -m "Cambio {contador}"

In [ ]:
#Inicializamos el objeto inicializador y configuramos los parámetros restantes
#Se crea el objeto init
init = init_param()
#Se obtienen los parámetros del archivo de configuración
params = init.get_params()
#Si no está inicializado el git se inicializa
init.init_git()
#Si no está inicializado el dvc se inicializa
init.init_dvc()
#Se incrementa run_count
init.update_params(params, "run_count")
#Se dan generales
!git config --global user.email "a01796238@tec.mx"
!git config --global user.name  "a01796238"


Repositorio Git ya inicializado.
Repositorio DVC ya inicializado.
Número de corrida actual para run_count: 3


In [ ]:
#Número de corridas
!git commit -m "Iteracion {current_run}..."
#Añadimos directorios
!dvc add data/raw
!git add data/raw.dvc
#Hacemos un push inicial
#!git push
#!dvc push

On branch master
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   run_params.json

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.config/
	data/.gitignore
	data/clean_data/.gitignore
	sample_data/

no changes added to commit (use "git add" and/or "git commit -a")
⠋ Checking graph
Adding...:   0% 0/1 [00:00<?, ?file/s{'info': ''}]
!
          |0.00 [00:00,     ?file/s]
                                    
!
  0% |          |0/? [00:00<?,    ?files/s]
                                           
!
Checking out /content/data/raw          |0.00 [00:00,    ?files/s]
Adding...: 100% 1/1 [00:00<00:00, 20.71file/s{'info': ''}]

To track the changes with git, run:

	git add data/raw.dvc

To enable auto staging, run:

	dvc config core.autostage true


In [ ]:
import os
from google.colab import files
# Definimos el directorio raw
DATA_RAW_DIR = "./data/raw"
# Definimos el diccionario
DICTIONARY_FILE = os.path.join(DATA_RAW_DIR, "dictionary.txt")
# Definimos nuestro archivo de salida
DATA_FILE = DATA_RAW_DIR + "/" + "insurance_company_modified.csv"

# El directorio existe? Sino lo creo...
if not os.path.exists(DATA_RAW_DIR):
    # If it doesn't exist, create it (including any parent directories)
    os.makedirs(DATA_RAW_DIR)
    print(f"Directory '{DATA_RAW_DIR}' created.")
else:
    #Si existe pero los archivos no existen...
    print(f"Directory '{DATA_RAW_DIR}' already exists.")
    #Los subimos
    if not os.path.exists(DICTIONARY_FILE):
        print(f"Directory '{DICTIONARY_FILE}' does not exist.")
        files.upload(DATA_RAW_DIR)
    if not os.path.exists(DATA_FILE):
        print(f"Directory '{DATA_FILE}' does not exist.")
        files.upload(DATA_RAW_DIR)


Directory './data/raw' already exists.


In [ ]:
# Este notebook contiene código para limpiar y procesar un dataset de seguros.
# Se documentarán las diferentes secciones y funciones para una mejor comprensión.

# Si no se está usando un ambiente virtual, descomentar y ejecutar las siguientes líneas
# Instalar la librería numpy si no está instalada
# !pip install numpy
# Instalar la librería pandas si no está instalada
# !pip install pandas
# Instalar la librería ucimlrepo si no está instalada (parece no usarse en el código actual, pero se mantiene por si acaso)
# !pip install ucimlrepo
# Instalar la librería seaborn si no está instalada (parece no usarse en el código actual, pero se mantiene por si acaso)
#!pip install seaborn

# Importar el módulo sys para acceder a funcionalidades del sistema
import sys
# Imprimir la ruta del ejecutable de Python que se está utilizando
print(sys.executable)

# Cargando las librerías necesarias para el procesamiento de datos y visualización

# Módulo para interactuar con el sistema operativo
import os
# Clase para trabajar con rutas de archivos de forma orientada a objetos
from pathlib import Path
# Módulo para operaciones con expresiones regulares
import re
# Librería para operaciones numéricas y arrays
import numpy as np
# Librería para manipulación y análisis de datos (DataFrames)
import pandas as pd
# Módulo para crear visualizaciones estáticas, interactivas y animadas en Python
import matplotlib.pyplot as plt
# Tipo de dato para columnas categóricas en pandas
from pandas.api.types import CategoricalDtype
# *Variable para controlar si se ignoran las advertencias
ignore_warnings = True
# *Librería para crear gráficos estadísticos atractivos
import seaborn as sns
# Módulo para trabajar con datos en formato JSON
import json
# *Función para obtener datasets del repositorio UCI
#from ucimlrepo import fetch_ucirepo
# *Módulo para trabajar con archivos YAML
#import yaml

# --- Configuración mínima del script ---
# Definir la ruta al archivo del diccionario de datos
DICT_TXT = Path(r"./data/raw/dictionary.txt")
# Definir la ruta al archivo CSV original modificado del dataset de la compañía de seguros
CSV_PATH = Path(r"./data/raw/insurance_company_modified.csv")
# Definir el directorio de salida para los datos limpios
OUT_DIR  = Path(r"./data/clean_data")
# Crear el directorio de salida si no existe (incluyendo directorios padres)
OUT_DIR.mkdir(parents=True, exist_ok=True)
# Definir la ruta completa del archivo CSV limpio de salida
OUT_CSV    = OUT_DIR / "coil2000_clean.csv"
# Definir la ruta completa del archivo de reporte de tipos de datos de salida
OUT_REPORT = OUT_DIR / "coil2000_dtypes_report.csv"
# Definir el número esperado de columnas en el dataset
N_COLS = 86
# Definir un conjunto con los nombres de las columnas que deben ser tratadas como categóricas
CAT_COLS = {"MOSTYPE", "MOSHOOFD", "MKOOPKLA"}

# Definir los rangos válidos para algunas columnas numéricas para proceso de outliers
RANGES = {
    "MOSTYPE":  (1, 41),  # Rango para el tipo de cliente
    "MAANTHUI": (1, 10),  # Rango para el número de casas
    "MGEMOMV":  (1, 6),   # Rango para el ingreso promedio
    "MGEMLEEF": (1, 6),   # Rango para la edad promedio
    "MOSHOOFD": (1, 10),  # Rango para el tipo principal de cliente
    "MGODRK":   (0, 9),   # Rango para la afiliación religiosa
    "PWAPART":  (0, 9),  # Rango para el número de pólizas de seguro privado
    "AWAPART":  (1, 12),  # Rango para el número de pólizas de seguro de coche
    "CARAVAN":  (0, 1),   # Rango para si tiene caravana este es nuestra variable categórica (0: No, 1: Sí)
}

# Definir un patrón de expresión regular para identificar valores inválidos o nulos en strings
INVALID_PATTERN = re.compile(r"(?i)^\s*$|^(nan|none|null|n/a|invalid|\?|unknown|error|missing|-)$")

class DataCleaner:
    def __init__(self, dict_path: Path, csv_path: Path, out_dir: Path, n_cols: int, cat_cols: set, ranges: dict, invalid_pattern: re.Pattern):
        self.dict_path = dict_path
        self.csv_path = csv_path
        self.out_dir = out_dir
        self.n_cols = n_cols
        self.cat_cols = cat_cols
        self.ranges = ranges
        self.invalid_pattern = invalid_pattern
        self.out_csv = self.out_dir / "coil2000_clean.csv"
        self.out_report = self.out_dir / "coil2000_dtypes_report.csv"

    # 1) Función para parsear el diccionario de datos y obtener los nombres de las columnas
    def parse_dictionary(self):
        txt = self.dict_path.read_text(encoding="utf-8", errors="ignore")
        names = []
        in_table = False
        for ln in txt.splitlines():
            ln = ln.strip()
            if re.match(r"^1\s+\S+", ln):
                in_table = True
            if not in_table:
                continue
            if not ln or ln.startswith("L0"):
                break
            m = re.match(r"^(\d+)\s+(\S+)\s+.*$", ln)
            if m:
                names.append((int(m.group(1)), m.group(2)))
        cols = [n for _, n in sorted(names, key=lambda x: x[0])]
        if len(cols) < self.n_cols:
            raise ValueError(f"El diccionario tiene {len(cols)} nombres; se requieren {self.n_cols}.")
        return cols[:self.n_cols]

    # 2) Función para limpiar cadenas de caracters y reemplazarlos por NaN
    def normalize_strings_to_nan(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        # Removed the call to init.get_params("limpio_count") as init is not defined in this class scope
        for c in df.select_dtypes(include="object").columns:
            df[c] = df[c].astype("string").str.strip()
        return df.replace(self.invalid_pattern, np.nan, regex=True)

    # 3) Función para castear los tipos de datos de las columnas
    def cast_types(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        non_cat = [c for c in df.columns if c not in self.cat_cols]
        df[non_cat] = df[non_cat].apply(pd.to_numeric, errors="coerce")
        for c in self.cat_cols:
            if c in df.columns:
                df[c] = df[c].astype("category")
        return df

    # 4) Función para aplicar reglas de validación por intervalos cerrados
    def apply_interval_rules(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        print(df.shape)

        for col, (lo, hi) in self.ranges.items():
            if col not in df.columns:
                continue
            s = pd.to_numeric(df[col].astype("string").str.strip(), errors="coerce") \
                if col in self.cat_cols or not pd.api.types.is_numeric_dtype(df[col]) else df[col]
            mask = s.between(lo, hi, inclusive="both")
            removed = (~mask).sum()
            if removed:
                print(f"{col} ∈ [{lo}, {hi}]: filas eliminadas = {removed}")
            df = df[mask].copy()
        if "CARAVAN" in df.columns:
            vals = set(pd.Series(df["CARAVAN"]).dropna().unique().tolist())
            if vals <= {0, 1}:
                df["CARAVAN"] = pd.to_numeric(df["CARAVAN"], errors="coerce").astype("Int64").astype("int64")
        df.reset_index(drop=True, inplace=True)
        return df

    # 5) Función para eliminar filas con valores nulos
    def enforce_no_nulls(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.replace(r"^\s*$", np.nan, regex=True)
        before = len(df)
        df = df.dropna().reset_index(drop=True)
        print(f"Filas eliminadas por nulos/vacíos: {before - len(df)}")
        return df

    # 6) Pipeline principal de limpieza y procesamiento de datos
    def clean_data(self):
        cols = self.parse_dictionary()

        df = pd.read_csv(self.csv_path, header=None)
        if df.shape[1] < self.n_cols:
            raise ValueError(f"El CSV tiene {df.shape[1]} columnas; se requieren {self.n_cols}.")
        df = df.iloc[:, :self.n_cols].copy()
        df.columns = cols

        df = self.normalize_strings_to_nan(df)
        df = self.apply_interval_rules(df)
        df = self.cast_types(df)
        df = self.enforce_no_nulls(df)
        assert df.isna().sum().sum() == 0

        df.to_csv(self.out_csv, index=False)
        init.commit(df, self.out_csv, "run_count")

        report = pd.DataFrame({
            "columna": df.columns,
            "dtype_final": df.dtypes.astype(str).values,
            "nulos": df.isna().sum().values,
            "unicos": df.nunique(dropna=True).values,
            "es_categorica": [c in self.cat_cols for c in df.columns],
        })
        report.to_csv(self.out_report, index=False)
        init.commit(report, self.out_report, "report_count")

        print(f"Guardado dataset limpio: {self.out_csv}")
        print(f"Guardado reporte dtypes: {self.out_report}")

# Ejecutar la función principal si el script se ejecuta directamente
if __name__ == "__main__":
    cleaner = DataCleaner(DICT_TXT, CSV_PATH, OUT_DIR, N_COLS, CAT_COLS, RANGES, INVALID_PATTERN)
    cleaner.clean_data()

/usr/bin/python3
(5938, 86)
MOSTYPE ∈ [1, 41]: filas eliminadas = 75
MAANTHUI ∈ [1, 10]: filas eliminadas = 67
MGEMOMV ∈ [1, 6]: filas eliminadas = 62
MGEMLEEF ∈ [1, 6]: filas eliminadas = 67
MOSHOOFD ∈ [1, 10]: filas eliminadas = 56
MGODRK ∈ [0, 9]: filas eliminadas = 26
PWAPART ∈ [0, 9]: filas eliminadas = 42
AWAPART ∈ [1, 12]: filas eliminadas = 2927
CARAVAN ∈ [0, 1]: filas eliminadas = 8
Filas eliminadas por nulos/vacíos: 1267
Segundo dato modificado guardado en data/clean_data/coil2000_clean.csv
⠋ Checking graph
Adding...:   0% 0/1 [00:00<?, ?file/s{'info': ''}]
!
          |0.00 [00:00,     ?file/s]
                                    
!
  0% |          |0/? [00:00<?,    ?files/s]
                                           
  0% 0/1 [00:00<?, ?files/s]
  0% 0/1 [00:00<?, ?files/s{'info': ''}]
Adding...: 100% 1/1 [00:00<00:00, 25.19file/s{'info': ''}]

To track the changes with git, run:

	git add data/clean_data/coil2000_clean.csv.dvc

To enable auto staging, run:

	dvc config co

In [ ]:
!git push

fatal: No configured push destination.
Either specify the URL from the command-line or configure a remote repository using

    git remote add <name> <url>

and then push using the remote name

    git push <name>

